# Data

In [ ]:
import numpy as np
from pathlib import Path


data_dir = Path("data")
imgs_dir = data_dir / "things-eeg2-imgs"
eeg_dir = data_dir / "things-eeg2-pre"


ex_data_path = "data/things-eeg2-pre/sub-01/preprocessed_eeg_training.npy"
ex_data = np.load(ex_data_path, allow_pickle=True).item()
ex_data.keys()

In [ ]:
train_img_paths = [
    img_dir
    for concept_dir in sorted((imgs_dir / "training_images").iterdir())
    for img_dir in sorted(concept_dir.iterdir())
]
test_img_paths = [
    img_dir
    for concept_dir in sorted((imgs_dir / "test_images").iterdir())
    for img_dir in sorted(concept_dir.iterdir())
]

train_imgs_per_concept = 10
test_imgs_per_concept = 1

In [ ]:
from typing import Literal


def idx_to_img_path(idx: int, split: Literal["train", "test"]) -> Path:
    if split == "train":
        paths = train_img_paths

    elif split == "test":
        paths = test_img_paths

    else:
        raise ValueError(f"Invalid split: {split}")

    return paths[idx]


print("0", idx_to_img_path(0, "train"))  # Aardvark 1
print("10", idx_to_img_path(10, "train"))  # Acabus 1
print("11", idx_to_img_path(11, "train"))  # Acabus 2

## Image Data

In [4]:
import torchvision
from torchvision.transforms import v2 as tv2
import matplotlib.pyplot as plt


def load_img(idx: int, split: Literal["train", "test"]):
    path = idx_to_img_path(idx, split)
    img = torchvision.io.decode_image(str(path))
    return img


img = load_img(0, "train")

print("Size:", img.size())
print("Dtype:", img.dtype)
print("Min", "Max:", img.min(), img.max())

tv2.functional.to_pil_image(img)

In [ ]:
import torch


def preprocess_img(img: torch.Tensor) -> torch.Tensor:
    """Preprocess image for by resizing to 256, then normalizing to [0,1]."""

    img = tv2.functional.resize(img, [256])
    img_pp = img / 255.0
    return img_pp


img_pp = preprocess_img(img)
print("Size:", img_pp.size())
print("Dtype:", img_pp.dtype)
print("Min", "Max:", img_pp.min(), img_pp.max())

plt.imshow(img_pp.permute(1, 2, 0).numpy())
plt.axis("off")
plt.show()

In [ ]:
img2_pp = load_img(11, "train")
img2_pp = preprocess_img(img2_pp)
plt.imshow(img2_pp.permute(1, 2, 0).numpy())
plt.axis("off")

In [ ]:
noisy_img_pp_05 = torch.clamp(img_pp + 0.5 * torch.randn_like(img_pp), 0, 1)
noisy_img_pp_005 = torch.clamp(img_pp + 0.05 * torch.randn_like(img_pp), 0, 1)
plt.imshow(noisy_img_pp_05.permute(1, 2, 0).numpy())
plt.axis("off")

In [ ]:
from dreamsim.model import dreamsim


dreamsim_model, _ = dreamsim(dreamsim_type="synclr_vitb16")
dreamsim_model.requires_grad_(False)
dreamsim_model.eval()

In [ ]:
sim_img_to_img2 = dreamsim_model(
    img_pp.unsqueeze(0).to(dreamsim_model.device),
    img2_pp.unsqueeze(0).to(dreamsim_model.device),
)
sim_img_to_noise_005 = dreamsim_model(
    img_pp.unsqueeze(0).to(dreamsim_model.device),
    noisy_img_pp_005.unsqueeze(0).to(dreamsim_model.device),
)
sim_img_to_noise_05 = dreamsim_model(
    img_pp.unsqueeze(0).to(dreamsim_model.device),
    noisy_img_pp_05.unsqueeze(0).to(dreamsim_model.device),
)

print("Dreamsim:")
print("\tSimilarity img to img2:", sim_img_to_img2.item())
print("\tSimilarity img to noise (0.05):", sim_img_to_noise_005.item())
print("\tSimilarity img to noise (0.5):", sim_img_to_noise_05.item())

In [ ]:
aligned_synclr_model = dreamsim_model.base_model.model.extractor_list[0].model
aligned_synclr_model

In [ ]:
from dreamsim.feature_extraction.load_synclr_as_dino import load_synclr_as_dino

synclr_model = load_synclr_as_dino(16)
synclr_model.requires_grad_(False)
synclr_model.eval()

In [ ]:
def get_cosine_sim(x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
    x1 = x1.flatten(start_dim=1)
    x2 = x2.flatten(start_dim=1)
    x1 = x1 / torch.norm(x1, dim=-1, keepdim=True)
    x2 = x2 / torch.norm(x2, dim=-1, keepdim=True)
    return (x1 @ x2.T).squeeze(0)

In [ ]:
z1 = synclr_model(img_pp.unsqueeze(0))
z2 = synclr_model(img2_pp.unsqueeze(0))
zn005 = synclr_model(noisy_img_pp_005.unsqueeze(0))
zn05 = synclr_model(noisy_img_pp_05.unsqueeze(0))

sim_img_to_img2 = get_cosine_sim(z1, z2)
sim_img_to_noise_005 = get_cosine_sim(z1, zn005)
sim_img_to_noise_05 = get_cosine_sim(z1, zn05)

print("SimClr:")
print("\tSimilarity img to img2:", sim_img_to_img2.item())
print("\tSimilarity img to noise (0.05):", sim_img_to_noise_005.item())
print("\tSimilarity img to noise (0.5):", sim_img_to_noise_05.item())

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

z1 = aligned_synclr_model(img_pp.unsqueeze(0).to(device))
z2 = aligned_synclr_model(img2_pp.unsqueeze(0).to(device))
zn005 = aligned_synclr_model(noisy_img_pp_005.unsqueeze(0).to(device))
zn05 = aligned_synclr_model(noisy_img_pp_05.unsqueeze(0).to(device))

sim_img_to_img2 = get_cosine_sim(z1, z2)
sim_img_to_noise_005 = get_cosine_sim(z1, zn005)
sim_img_to_noise_05 = get_cosine_sim(z1, zn05)

print("Aligned SimClr:")
print("\tSimilarity img to img2:", sim_img_to_img2.item())
print("\tSimilarity img to noise (0.05):", sim_img_to_noise_005.item())
print("\tSimilarity img to noise (0.5):", sim_img_to_noise_05.item())

In [ ]:
import tqdm

GEN_EMBEDDINGS = False


def gen_embeddings(
    model: torch.nn.Module,
    img_paths: list[Path],
    split: Literal["train", "test"],
    batch_size: int = 32,
    device: torch.device = device,
) -> np.ndarray:
    """Generate embeddings for a given split of images."""
    model.eval()
    model.to(device)
    embeddings = []
    with torch.no_grad():
        for i in tqdm.tqdm(range(0, len(img_paths), batch_size)):
            imgs = [
                load_img(j, split=split)
                for j in range(i, min(i + batch_size, len(img_paths)))
            ]
            imgs = [preprocess_img(img).unsqueeze(0) for img in imgs]
            imgs = torch.cat(imgs, dim=0).to(device)
            emb = model(imgs)
            embeddings.append(emb.cpu().numpy())
    return np.concatenate(embeddings, axis=0)


embed_dir = data_dir / "things-eeg2-imgs-embed"
for model, name in [
    (synclr_model, "synclr"),
    (aligned_synclr_model, "aligned_synclr"),
]:
    if GEN_EMBEDDINGS:
        print(f"Generating {name} embeddings...")

        train_embeddings = gen_embeddings(
            model, train_img_paths, split="train", batch_size=512
        )
        np.save(f"{embed_dir / name}/train_embeddings.npy", train_embeddings)

        test_embeddings = gen_embeddings(
            model, test_img_paths, split="test", batch_size=512
        )
        np.save(f"{embed_dir / name}/test_embeddings.npy", test_embeddings)

    else:
        print("Loading embeddings...")

        train_embeddings = np.load(f"{embed_dir / name}/train_embeddings.npy")
        test_embeddings = np.load(f"{embed_dir / name}/test_embeddings.npy")

## EEG Data

In [ ]:
channel_names = ex_data["ch_names"]
print(f"Channel names: {', '.join(channel_names)}")

eeg_data = ex_data["preprocessed_eeg_data"]
print("EEG data: ")
print(f"\tShape: {eeg_data.shape}")
print(f"\tData type: {eeg_data.dtype}")
print(f"\tMaximum value: {np.max(eeg_data)}")
print(f"\tMinimum value: {np.min(eeg_data)}")
print(f"\tMean value: {np.mean(eeg_data)}")
print(f"\tStandard deviation: {np.std(eeg_data)}")

times = ex_data["times"]
print(f"Times: {times}")

In [ ]:
def preprocess_eeg_data(eeg_data):
    """Preprocess the EEG data by averaging over the number of repetitions.

    Args:
        eeg_data (numpy.ndarray): The EEG data to preprocess. <concepts, repetitions, channels, timesteps>

    Returns:
        numpy.ndarray: The preprocessed EEG data. <concepts, channels, timesteps>
    """
    # Average over the number of repetitions
    preprocessed_data = np.mean(eeg_data, axis=1).astype(np.float32)
    return preprocessed_data


eeg_data_pp = preprocess_eeg_data(eeg_data)
eeg_data_pp.shape

In [ ]:
def load_eeg_data(eeg_path: Path):
    """Load EEG data from a given path."""
    eeg_data = np.load(eeg_path, allow_pickle=True).item()
    return (
        preprocess_eeg_data(eeg_data["preprocessed_eeg_data"]),
        eeg_data["times"],
        eeg_data["ch_names"],
    )


def load_all_eeg_data(
    eeg_paths: list[Path],
) -> tuple[torch.Tensor, torch.Tensor, list[str]]:
    all_eeg_data = []
    all_times = None
    all_ch_names = []

    for eeg_path in eeg_paths:
        eeg_data, times, ch_names = load_eeg_data(eeg_path)
        all_eeg_data.append(torch.from_numpy(eeg_data))

        if all_times is None:
            all_times = torch.tensor(times)

        if not all_ch_names:
            all_ch_names = ch_names

    if all_times is None:
        all_times = torch.tensor([])

    return torch.concat(all_eeg_data), all_times, all_ch_names

## Unified Data

In [ ]:
from pydantic import BaseModel
from torch.utils.data import Dataset


class EEGDatasetConfig(BaseModel):
    data_path: Path = Path("data")
    imgs_dir: str = "things-eeg2-imgs"
    eeg_dir: str = "things-eeg2-pre"
    imgs_latent_dir: str = "things-eeg2-imgs-embed"
    train_imgs_per_concept: int = 10
    test_imgs_per_concept: int = 1
    subs: list[int] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


class EEGDataset(Dataset):
    def __init__(
        self,
        config: EEGDatasetConfig,
        split: Literal["train", "test"],
        model_name: str = "synclr",
    ):
        self.config = config
        self.split = split
        self.model_name = model_name
        self.imgs_per_concepts = (
            self.config.train_imgs_per_concept
            if split == "train"
            else self.config.test_imgs_per_concept
        )

        if split == "train":
            img_subdir = "training_images"
            img_embed_name = "train_embeddings"
            eeg_name = "preprocessed_eeg_training"

        else:
            img_subdir = "test_images"
            img_embed_name = "test_embeddings"
            eeg_name = "preprocessed_eeg_test"

        self.img_paths = [
            img_dir
            for concept_dir in sorted(
                (self.config.data_path / self.config.imgs_dir / img_subdir).iterdir()
            )
            for img_dir in sorted(concept_dir.iterdir())
        ]
        self.img_latents = torch.from_numpy(
            np.load(
                self.config.data_path
                / f"{self.config.imgs_latent_dir}/{model_name}/{img_embed_name}.npy"
            )
        ).float()

        self.eeg_data_paths = [
            self.config.data_path
            / self.config.eeg_dir
            / f"sub-{sub:02}"
            / f"{eeg_name}.npy"
            for sub in self.config.subs
        ]
        self.eeg_data, self.times, self.ch_names = load_all_eeg_data(
            self.eeg_data_paths
        )

    def __len__(self):
        return len(self.eeg_data)

    def __getitem__(self, idx: int):
        img_idx = idx % (
            len(self.img_paths)
        )  # EEG has stacked over subs, so we need to find the right sample within the sub

        img_path = self.img_paths[img_idx]
        img_latent = self.img_latents[img_idx]

        eeg_data = self.eeg_data[idx]

        return {
            "img_path": str(img_path),
            "img_latent": img_latent,
            "eeg_data": eeg_data,
        }

In [ ]:
from typing import cast
from torch.utils.data import random_split
from torch import manual_seed

config = EEGDatasetConfig()


def make_datasets(
    config: EEGDatasetConfig,
    model: str = "synclr",
    seed: int = 42,
    train_val_split: float = 0.8,
) -> tuple[EEGDataset, EEGDataset, EEGDataset]:
    train_dataset = EEGDataset(config, split="train", model_name=model)
    test_dataset = EEGDataset(config, split="test", model_name=model)

    split_rng = manual_seed(seed)
    train_dataset, val_dataset = random_split(
        train_dataset, [train_val_split, 1 - train_val_split], generator=split_rng
    )
    train_dataset = cast(EEGDataset, train_dataset)
    val_dataset = cast(EEGDataset, val_dataset)
    test_dataset = cast(EEGDataset, test_dataset)

    return train_dataset, val_dataset, test_dataset


#train_dataset, val_dataset, test_dataset = make_datasets(
#    config, model="synclr", seed=42
#)

# Model

In [ ]:
# Adapted from https://github.com/eeyhsong/NICE-EEG
import torch
import torch.nn as nn
import einops
from einops.layers.torch import Rearrange


class EEGEncoderConfig(BaseModel):
    embed_dim: int = 40
    encoded_dim: int = 1440  # Result of embed dim * final spatial * final temporal
    proj_dim: int = 768

    temporal_kernel_size: int = 25
    spatial_kernel_size: int = 17
    temporal_pool_size: int = 41
    temporal_stride: int = 1
    hidden_dim: int = 40
    dropout: float = 0.5


class DebugLayer(nn.Module):
    def __init__(self, note: str = ""):
        super(DebugLayer, self).__init__()
        self.note = note

    def forward(self, x):
        print(f"(debug): {x.shape} - {self.note}")
        return x


class PatchEmbedding(nn.Module):
    def __init__(
        self,
        embed_dim,
        temporal_kernel_size,
        spatial_kernel_size,
        temporal_pool_size,
        temporal_stride,
        hidden_dim,
        dropout,
    ):
        super(PatchEmbedding, self).__init__()

        self.spatiotemporal_conv = nn.Sequential(
            nn.Conv2d(1, hidden_dim, kernel_size=(1, temporal_kernel_size)),
            nn.AvgPool2d(
                kernel_size=(1, temporal_pool_size), stride=(1, temporal_stride)
            ),
            nn.BatchNorm2d(hidden_dim),
            nn.ELU(),
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=(spatial_kernel_size, 1)),
            nn.BatchNorm2d(hidden_dim),
            nn.ELU(),
            nn.Dropout(dropout),
        )

        self.projection = nn.Sequential(
            nn.Conv2d(hidden_dim, embed_dim, kernel_size=(1, 1)),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = einops.rearrange(x, "b s t -> b 1 s t")
        x = self.spatiotemporal_conv(x)
        x = self.projection(x)
        x = einops.rearrange(x, "b e (s) (t) -> b (s t) e")
        return x


class EEGEncoder(nn.Module):
    def __init__(
        self,
        config: EEGEncoderConfig = EEGEncoderConfig(),
    ):
        super(EEGEncoder, self).__init__()

        self.patch_embedding = PatchEmbedding(
            embed_dim=config.embed_dim,
            temporal_kernel_size=config.temporal_kernel_size,
            spatial_kernel_size=config.spatial_kernel_size,
            temporal_pool_size=config.temporal_pool_size,
            temporal_stride=config.temporal_stride,
            hidden_dim=config.hidden_dim,
            dropout=config.dropout,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.patch_embedding(x)
        x = x.flatten(start_dim=1)

        return x


class LatentProjector(nn.Module):
    def __init__(
        self,
        embed_dim: int = 1440,
        proj_dim: int = 768,
        dropout: float = 0.5,
    ):
        super(LatentProjector, self).__init__()

        self.l_proj = nn.Linear(embed_dim, proj_dim)
        self.l_inner = nn.Sequential(
            nn.GELU(),
            nn.Linear(proj_dim, proj_dim),
            nn.Dropout(dropout),
        )
        self.norm = nn.LayerNorm(proj_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_res = x = self.l_proj(x)
        x = self.l_inner(x) + x_res
        x = self.norm(x)

        return x


eeg_config = EEGEncoderConfig()
eeg_encoder = EEGEncoder(eeg_config).to(device)

#ex_data = train_dataset[0]["eeg_data"].unsqueeze(0).to(device).to(torch.float32)
#eeg_encoder(ex_data).shape

In [ ]:
import math
from lightning import LightningModule


class NICEConfig(BaseModel):
    project_dim: int = 768
    eeg_latent_dim: int = 1440
    img_latent_dim: int = 768

    batch_size: int = 1024
    eval_batch_size: int = 200
    learning_rate: float = 3e-4
    beta1: float = 0.9
    beta2: float = 0.999
    num_workers: int = 4


class NICEModel(LightningModule):
    def __init__(
        self,
        eeg_config: EEGEncoderConfig | None = EEGEncoderConfig(),
        eeg_projector: LatentProjector | None = None,
        img_projector: LatentProjector | None = None,
        config: NICEConfig = NICEConfig(),
        dataset_config: EEGDatasetConfig = EEGDatasetConfig(),
        model_name: str = "synclr",
        data_seed: int = 42,
        eeg_encoder: EEGEncoder | None = None,
    ):
        super(NICEModel, self).__init__()

        assert (
            eeg_config or eeg_encoder
        ), "Either eeg_config or eeg_encoder must be provided."

        self.config = config
        self.eeg_encoder = eeg_encoder or EEGEncoder(eeg_config)  # type: ignore #
        self.eeg_projector = eeg_projector or LatentProjector(
            embed_dim=config.eeg_latent_dim,
            proj_dim=config.project_dim,
        )
        self.img_projector = img_projector or LatentProjector(
            embed_dim=config.img_latent_dim,
            proj_dim=config.project_dim,
        )
        self.loss = nn.CrossEntropyLoss()

        self.train_dataset, self.val_dataset, self.test_dataset = make_datasets(
            dataset_config,
            model=model_name,
            seed=data_seed,
        )
        self.model_name = model_name

        self.save_hyperparameters(
            "eeg_encoder",
            "eeg_projector",
            "img_projector",
            "config",
            "dataset_config",
            "model_name",
            "data_seed",
        )

    def train_dataloader(self):
        """Return the training dataloader."""
        return (
            torch.utils.data.DataLoader(
                self.train_dataset,
                batch_size=self.config.batch_size,
                shuffle=True,
                num_workers=self.config.num_workers,
                pin_memory=True,
            )
            if self.train_dataset
            else None
        )

    def val_dataloader(self):
        """Return the validation dataloader."""
        return (
            torch.utils.data.DataLoader(
                self.val_dataset,
                batch_size=self.config.eval_batch_size,
                shuffle=False,
                num_workers=self.config.num_workers,
                pin_memory=True,
            )
            if self.val_dataset
            else None
        )

    def test_dataloader(self):
        """Return the test dataloader."""
        return (
            torch.utils.data.DataLoader(
                self.test_dataset,
                batch_size=self.config.eval_batch_size,
                shuffle=False,
                num_workers=self.config.num_workers,
                pin_memory=True,
            )
            if self.test_dataset
            else None
        )

    def forward(self, img_latent: torch.Tensor, eeg_data: torch.Tensor) -> torch.Tensor:
        """Forward pass through the model."""
        # Encode EEG data
        eeg_latent = self.eeg_encoder(eeg_data)
        eeg_latent = self.eeg_projector(eeg_latent)

        # Project image latent
        img_latent = self.img_projector(img_latent)

        # Compute similarity
        sim = eeg_latent @ img_latent.T 

        return sim

    def configure_optimizers(self):
        """Configure optimizers for the model."""
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.config.learning_rate,
            betas=(self.config.beta1, self.config.beta2),
        )
        return optimizer

    def get_loss(self, sim: torch.Tensor) -> torch.Tensor:
        """Compute cross-entropy loss."""
        labels = torch.arange(sim.size(0), device=sim.device)
        loss_e = self.loss(sim, labels)
        loss_i = self.loss(sim.T, labels)
        loss = (loss_e + loss_i) / 2
        return loss

    def get_top_n_accuracy(self, sim: torch.Tensor, n: int = 1) -> float:
        """Compute top-n accuracy."""
        labels = torch.arange(sim.size(0), device=sim.device)
        top_n = sim.topk(n, dim=-1).indices
        correct = (top_n == labels.unsqueeze(1)).float()
        return (correct.sum() / correct.numel()).item()

    def training_step(self, batch, batch_idx):
        """Training step for the model."""
        img_latent = batch["img_latent"]
        eeg_data = batch["eeg_data"]

        sim = self(img_latent, eeg_data)
        loss = self.get_loss(sim)

        self.log("train/loss", loss, prog_bar=True, on_step=True, on_epoch=False)
        return loss

    def validation_step(self, batch, batch_idx):
        img_latent = batch["img_latent"]
        eeg_data = batch["eeg_data"]

        sim = self(img_latent, eeg_data)
        loss = self.get_loss(sim)

        top1_acc = self.get_top_n_accuracy(sim, n=1)
        top3_acc = self.get_top_n_accuracy(sim, n=3)
        top5_acc = self.get_top_n_accuracy(sim, n=5)

        self.log("val/loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("val/top1_acc", top1_acc, prog_bar=False, on_step=False, on_epoch=True)
        self.log("val/top3_acc", top3_acc, prog_bar=False, on_step=False, on_epoch=True)
        self.log("val/top5_acc", top5_acc, prog_bar=False, on_step=False, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        img_latent = batch["img_latent"]
        eeg_data = batch["eeg_data"]

        sim = self(img_latent, eeg_data)
        loss = self.get_loss(sim)
        top1_acc = self.get_top_n_accuracy(sim, n=1)
        top3_acc = self.get_top_n_accuracy(sim, n=3)
        top5_acc = self.get_top_n_accuracy(sim, n=5)

        self.log("test/loss", loss, prog_bar=True, on_step=False, on_epoch=False)
        self.log("test/top1_acc", top1_acc, prog_bar=True, on_step=False, on_epoch=True)
        self.log(
            "test/top3_acc", top3_acc, prog_bar=False, on_step=False, on_epoch=True
        )
        self.log(
            "test/top5_acc", top5_acc, prog_bar=False, on_step=False, on_epoch=True
        )
        return loss

# Train

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

def setup_and_train(
    config: NICEConfig = NICEConfig(),
    dataset_config: EEGDatasetConfig = EEGDatasetConfig(),
    model_name: str = "synclr",
    data_seed: int = 42,
    epochs: int = 100,
    save_top_k: int = 1,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = NICEModel(
        config=config,
        dataset_config=dataset_config,
        model_name=model_name,
        data_seed=data_seed,
    ).to(device)

    checkpoint_callback = ModelCheckpoint(
        monitor="val/loss",
        filename="checkpoint/{epoch:02d}-{val/loss:.2f}",
        save_top_k=save_top_k,
        mode="min",
    )

    logger = TensorBoardLogger(
        save_dir="logs",
        name=model_name,
        default_hp_metric=False,
    )

    trainer = Trainer(
        max_epochs=epochs,
        callbacks=[checkpoint_callback],
        logger=logger,
        enable_progress_bar=True,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
    )

    trainer.fit(model)
    return model


In [ ]:
TRAIN_SYNCLR = True

if TRAIN_SYNCLR:
    setup_and_train(
        model_name="synclr",
    )

In [ ]:
TRAIN_ALIGNED_SYNCLR = True

if TRAIN_ALIGNED_SYNCLR:
    setup_and_train(
        model_name="aligned_synclr",
    )